# ⚡ Power Procurement Strategy ML Pipeline - Demo Notebook

This notebook demonstrates the complete ML pipeline for power procurement strategy recommendation in AI data centers.

## Pipeline Overview
1. **Data Generation** - Create synthetic training data
2. **Power Forecasting** - Predict power demand from GPU utilization
3. **Strategy Recommendation** - Recommend procurement mix (PPA, Spot, Battery)
4. **Evaluation** - Assess model performance

In [ ]:
# Install dependencies if needed
# !pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('dark_background')
sns.set_palette('husl')

# Import our modules
import sys
sys.path.insert(0, 'src')

from data_generator import PowerDataGenerator
from forecaster import PowerForecaster
from recommender import ProcurementRecommender
from evaluate import PipelineEvaluator

## 1. 📁 Data Generation

Generate synthetic dataset with realistic AI data center characteristics.

In [ ]:
# Initialize data generator
generator = PowerDataGenerator('config.yaml')

# Generate dataset
df = generator.generate_dataset(n_samples=500, save_path='data/synthetic_dataset.csv')

print(f"Generated {len(df)} samples")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Explore data distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# GPU distribution
axes[0, 0].hist(df['num_gpus'], bins=20, edgecolor='white', alpha=0.7)
axes[0, 0].set_xlabel('Number of GPUs')
axes[0, 0].set_title('GPU Count Distribution')

# GPU type
df['gpu_type'].value_counts().plot(kind='bar', ax=axes[0, 1], color=['#2ecc71', '#e74c3c', '#3498db'])
axes[0, 1].set_title('GPU Type Distribution')
axes[0, 1].tick_params(axis='x', rotation=0)

# Workload type
df['workload_type'].value_counts().plot(kind='bar', ax=axes[0, 2], color=['#9b59b6', '#f39c12'])
axes[0, 2].set_title('Workload Type Distribution')
axes[0, 2].tick_params(axis='x', rotation=0)

# Burst peak power
axes[1, 0].hist(df['burst_peak_kw'], bins=30, edgecolor='white', alpha=0.7, color='#e74c3c')
axes[1, 0].set_xlabel('Burst Peak (kW)')
axes[1, 0].set_title('Peak Power Distribution')

# Renewable target
axes[1, 1].hist(df['renewable_target_pct'], bins=20, edgecolor='white', alpha=0.7, color='#2ecc71')
axes[1, 1].set_xlabel('Renewable Target (%)')
axes[1, 1].set_title('Renewable Target Distribution')

# Contract lead time
axes[1, 2].hist(df['contract_lead_time_months'], bins=20, edgecolor='white', alpha=0.7, color='#3498db')
axes[1, 2].set_xlabel('Lead Time (months)')
axes[1, 2].set_title('Contract Lead Time Distribution')

plt.tight_layout()
plt.savefig('outputs/data_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Visualize sample GPU utilization and power traces
import json

fig, axes = plt.subplots(3, 2, figsize=(14, 12))

for i in range(3):
    sample = df.iloc[i]
    
    # Parse JSON if needed
    util = json.loads(sample['gpu_util_profile']) if isinstance(sample['gpu_util_profile'], str) else sample['gpu_util_profile']
    power = json.loads(sample['power_trace_kw']) if isinstance(sample['power_trace_kw'], str) else sample['power_trace_kw']
    
    t = np.arange(48)
    
    # Utilization
    axes[i, 0].plot(t, util, 'b-', linewidth=2)
    axes[i, 0].fill_between(t, util, alpha=0.3)
    axes[i, 0].set_ylabel('Utilization')
    axes[i, 0].set_title(f'Sample {i+1}: {sample["workload_type"]} ({sample["num_gpus"]} x {sample["gpu_type"]})')
    axes[i, 0].set_ylim(0, 1)
    
    # Power
    axes[i, 1].plot(t, power, 'r-', linewidth=2)
    axes[i, 1].fill_between(t, power, alpha=0.3, color='red')
    axes[i, 1].set_ylabel('Power (kW)')
    axes[i, 1].set_title(f'Power Trace - Peak: {np.max(power):.1f} kW')

axes[2, 0].set_xlabel('Hour')
axes[2, 1].set_xlabel('Hour')

plt.tight_layout()
plt.savefig('outputs/sample_traces.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. 🧠 Power Demand Forecasting

Train LSTM/TCN model to predict power traces from GPU utilization.

In [ ]:
# Split data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")

In [ ]:
# Train forecaster
forecaster = PowerForecaster('config.yaml')
history = forecaster.train(train_df, verbose=True)

In [ ]:
# Plot training history
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history['train_loss'], label='Training Loss', linewidth=2)
ax.plot(history['val_loss'], label='Validation Loss', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (MSE)')
ax.set_title('Forecaster Training History')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/forecaster_training.png', dpi=150)
plt.show()

In [ ]:
# Evaluate forecaster
fc_metrics = forecaster.evaluate(test_df)
print("\n📊 Forecaster Metrics:")
for k, v in fc_metrics.items():
    print(f"  {k}: {v}")

In [ ]:
# Visualize predictions
gpu_util, power_traces, metadata = forecaster.prepare_data(test_df.head(5), True)
predictions = forecaster.predict(gpu_util, metadata)
actual = forecaster.power_scaler.inverse_transform(power_traces.reshape(-1, 1)).reshape(power_traces.shape)

fig, axes = plt.subplots(3, 1, figsize=(12, 10))
t = np.arange(48)

for i, ax in enumerate(axes):
    ax.plot(t, actual[i], 'b-', label='Actual', linewidth=2)
    ax.plot(t, predictions[i], 'r--', label='Predicted', linewidth=2)
    ax.fill_between(t, actual[i], predictions[i], alpha=0.3, color='gray')
    ax.set_xlabel('Hour')
    ax.set_ylabel('Power (kW)')
    ax.set_title(f'Sample {i+1}: Actual vs Predicted Power Trace')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/forecaster_predictions.png', dpi=150)
plt.show()

## 3. 🎯 Procurement Strategy Recommender

Train XGBoost model to recommend procurement mix.

In [ ]:
# Train recommender
recommender = ProcurementRecommender('config.yaml')
rec_metrics = recommender.train(train_df, verbose=True)

In [ ]:
# Feature importance
importance_df = recommender.feature_importance()

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=importance_df.head(15), x='importance', y='feature', ax=ax, palette='viridis')
ax.set_xlabel('Importance')
ax.set_ylabel('Feature')
ax.set_title('Top 15 Features by Importance')
plt.tight_layout()
plt.savefig('outputs/feature_importance.png', dpi=150)
plt.show()

In [ ]:
# Test predictions
predictions = recommender.predict(test_df)
predictions.head(10)

In [ ]:
# Visualize procurement mix
evaluator = PipelineEvaluator()
evaluator.plot_procurement_mix(predictions, save_path='outputs/procurement_mix.png')

In [ ]:
# Cost analysis
evaluator.plot_cost_analysis(predictions, save_path='outputs/cost_analysis.png')

## 4. 📋 Sample Prediction Demo

Demonstrate end-to-end prediction for a new cluster.

In [ ]:
# Create a sample cluster configuration
sample_cluster = {
    'num_gpus': 1024,
    'gpu_type': 'H100',
    'rack_density_kw': 30.0,
    'workload_type': 'LLM_Training',
    'batch_size': 64,
    'seq_length': 4096,
    'fp_precision': 'bf16',
    'gpu_util_profile': [0.9 + np.random.normal(0, 0.03) for _ in range(48)],
    'power_trace_kw': [500 + np.random.normal(0, 20) for _ in range(48)],
    'burst_peak_kw': 650.0,
    'grid_capacity_mw': 100.0,
    'renewable_target_pct': 85.0,
    'onsite_gen_allowed': False,
    'ppa_price_usd_mwh': 55.0,
    'spot_price_avg_usd_mwh': 70.0,
    'battery_cost_usd_kwh': 180.0
}

# Get recommendation
result = recommender.predict_single(sample_cluster)

print("\n🎯 Procurement Recommendation for New Cluster")
print("=" * 50)
print(f"\nCluster: {sample_cluster['num_gpus']} x {sample_cluster['gpu_type']}")
print(f"Workload: {sample_cluster['workload_type']}")
print(f"\n📊 Recommended Mix:")
for k, v in result['recommended_mix'].items():
    print(f"  {k.upper()}: {v} MW")
print(f"\n📈 Mix Percentages:")
for k, v in result['mix_percentages'].items():
    print(f"  {k.upper()}: {v}%")
print(f"\n💰 Forecasted Cost: ${result['forecasted_cost_usd_mwh']}/MWh")
print(f"📅 Contract Lead Time: {result['contract_lead_time_months']} months")

## 5. 💾 Save Models

Save trained models for deployment.

In [ ]:
import os
os.makedirs('models/forecaster', exist_ok=True)
os.makedirs('models/recommender', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

# Save models
forecaster.save('models/forecaster')
recommender.save('models/recommender')

print("\n✅ Models saved successfully!")

In [ ]:
# Generate evaluation report
report = evaluator.generate_report(
    fc_metrics,
    rec_metrics,
    predictions,
    save_path='outputs/evaluation_report.md'
)

print(report)

## 📝 Summary

This notebook demonstrated:

1. ✅ **Data Generation**: Created 500 synthetic samples with realistic AI workload characteristics
2. ✅ **Power Forecasting**: Trained LSTM model to predict power demand from GPU utilization
3. ✅ **Strategy Recommendation**: Built XGBoost model to recommend PPA/Spot/Battery mix
4. ✅ **Evaluation**: Computed metrics and generated visualizations

### Next Steps
- Run `streamlit run app.py` for interactive dashboard
- Train RL optimizer with `python src/rl_optimizer.py`
- Integrate with real telemetry data